In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import re
import bar_chart_race as bcr

import warnings
warnings.filterwarnings("ignore")

# Items a fazer:
3 - Maior aparição em cenas diferentes em uma cena só

4 - Relação número de falas de personagem x n de views

5 - Quanto menos personagem, melhor a nota?

6 - Quanto menos ou mais cenas, melhor a nota

7 - Maior fala

8 - Menor fala

9 - Maior cena

10 - Mais tempo de episódio - melhor?

11 - Roteirista mais falas?

12 - Diretor mais falas?

13 - N de falas por temporada, eps

14 - Quantos compartilharam a mesma cena?

15 - Quem começo mais os eps? E terminou?

16 - Lista de palavras que mais apareceram escolhidas pelo usuário

In [2]:
async def read_data(file_path):
    return pd.read_excel(f'data/output_data/{file_path}.xlsx')

eps = await read_data('eps')
lines = await read_data('lines')
# - 11 segundos :)

# 1 - Qual o nome mais falado?

In [4]:
async def gerar_video(lines):
    personagens_principais = ['Michael', 'Dwight', 'Jim', 'Pam', 'Ryan', 'Andy',
                            'Stanley', 'Phyllis', 'Creed', 'Meredith', 'Darryl',
                            'Angela', 'Oscar', 'Kevin',
                            'Toby', 'Kelly', 'Erin']

    ref = {
        'tempisodio':[],
        'nome':[],
        'qtd':[]
    }

    for pp in personagens_principais:
        for t in eps['temporada'].unique().tolist():
            filtrado = lines[lines['temporada']==t]
            for e in filtrado['episodio'].unique().tolist():
                filtrado_e = filtrado[filtrado['episodio']==e]
                qtd = filtrado_e['fala'].str.contains(pp).sum()
                ref['tempisodio'].append('Temporada ' + str(t) + ' Episódio ' + str(e))
                ref['nome'].append(pp)
                ref['qtd'].append(qtd)

    r_1 = pd.DataFrame(ref)
    r_1 = pd.pivot_table(r_1, values='qtd', index='tempisodio', columns='nome')

    for pp in personagens_principais:
        r_1[pp] = r_1[pp].cumsum()
        
    # r_1 = r_1.head(10)

    title = 'Qual o nome mais mencionado na série?'
    bcr.bar_chart_race(df=r_1, filename='data/output_data/qual_nome_mais_falado.mp4', orientation='h', sort='desc',
                        fixed_order=False, fixed_max=True, title=title, steps_per_period=60, period_length=250
    )
    print(f'Video salvo - {title}')
await gerar_video(lines)

Video salvo - Qual o nome mais mencionado na série?


# 2 - Quantas vezes personagem x falou o nome de y?

In [58]:
ref = {
    'quem_falou':[],
    'nome_falado':[],
    'qtd':[],
    'falando_proprio_nome':[]
}
async def personagem_x_falou_nome_y_geral(lines):
    personagens_principais = ['Michael', 'Dwight', 'Jim', 'Pam', 'Ryan', 'Andy',
                            'Stanley', 'Phyllis', 'Creed', 'Meredith', 'Darryl',
                            'Angela', 'Oscar', 'Kevin',
                            'Toby', 'Kelly', 'Erin']
    
    async def quem_falou_quem(quem_falou, nome_falado):
        all_falas = []

        cut = lines[lines['personagem']==quem_falou]
        if cut.shape[0] <= 0:
            raise Exception('Nome não consta na base de dados')
        cut_l = cut['fala'].tolist()
        for f in cut_l:
            fala = re.sub('[^a-zA-Z]+', ' ', f).strip()
            for f_ in fala.split(' '):
                all_falas.append(f_.lower())

        counter = Counter(all_falas)
        palavras_qtd = pd.DataFrame(counter, index=[0])
        palavras_qtd = pd.melt(palavras_qtd, value_vars=palavras_qtd.columns.tolist())
        palavras_qtd.rename(columns={'variable': 'palavra', 'value': 'qtd'}, inplace=True)
        try:
            q_x_falou = palavras_qtd[palavras_qtd['palavra']==nome_falado.lower()]['qtd'].values[0]
        except:
            q_x_falou = 0

        return quem_falou, nome_falado, q_x_falou
    
    for quem_falou in personagens_principais:
        for nome_falado in personagens_principais:
            q_f, n_f, q_x_f = await quem_falou_quem(quem_falou=quem_falou, nome_falado=nome_falado)
            ref['quem_falou'].append(q_f)
            ref['nome_falado'].append(n_f)
            ref['qtd'].append(q_x_f)
            ref['falando_proprio_nome'].append(1 if quem_falou == nome_falado else 0)

    return ref
dic = await personagem_x_falou_nome_y_geral(lines)
r_2 = pd.DataFrame(dic).to_excel('data/output_data/pergunta_2.xlsx', index=False)

# 3 - Maior aparição em cenas diferentes em uma cena só

In [46]:
temporada = lines['temporada'].unique().tolist()

ref = {
    'temporada':[],
    'episodio':[],
    'qtd_cena':[],
    'nº de falas':[],
    'nº de palavras':[],
    'nº de personagens dis':[],
    'lista_personagens':[]
}


for t in temporada:
        filt = lines[lines['temporada']==t]
        episodeos = filt['episodio'].unique().tolist()
        
        for e in episodeos:
            ult = filt[filt['episodio']==e]
            
            ref['temporada'].append(t)
            ref['episodio'].append(e)
            ref['qtd_cena'].append(ult['cena'].max())
            ref['nº de falas'].append(ult['fala'].count())
            ref['nº de palavras'].append(ult['n_palavras'].sum())
            ref['nº de personagens dis'].append(len(ult['personagem'].unique().tolist()))
            ref['lista_personagens'].append(str(ult['personagem'].unique().tolist())[1:-1])

df = pd.DataFrame(ref)
df.head()

,temporada,episodio,qtd_cena,nº de falas,nº de palavras,nº de personagens dis,lista_personagens
0,1,1,39,228,2756,14,"'Michael', 'Jim', 'Pam', 'Dwight', 'Jan', 'Tod..."
1,1,2,46,217,3015,12,"'Michael', 'Mr. Brown', 'Jim', 'Dwight', 'Pam'..."
2,1,3,39,244,2768,13,"'Michael', 'Pam', 'Jan', 'Jim', 'Dwight', 'Sta..."
3,1,4,56,265,3163,14,"'Dwight', 'Michael', 'Stanley', 'Pam', 'Phylli..."
4,1,5,50,235,2551,20,"'Michael', 'Dwight', 'Jim', 'Phyllis', 'Pam', ..."


In [49]:
eps = eps.merge(df, on=['temporada', 'episodio'], suffixes=('', '_delme'))
eps = eps[[c for c in eps.columns if not c.endswith('_delme')]]
eps = eps[[c for c in eps.columns if not c.endswith('Unna')]]
eps.head()

,temporada,episodio,episodeo_num_geral,titulo,descricao,dirigido_por,escrito_por,data_de_estreia,codigo,telespectadores_USA,...,duração,Date,estrelas_convidadas,Director,Writers,qtd_cena,nº de falas,nº de palavras,nº de personagens dis,lista_personagens
0,1,1,1,Pilot,The premiere episode introduces the boss and s...,Ken Kwapis,Ricky Gervais & Stephen Merchant and Greg Daniels,2005-03-24,1001,11200000,...,23,24 March 2005,NaN,Ken Kwapis,Ricky Gervais |Stephen Merchant and Greg Daniels,39,228,2756,14,"'Michael', 'Jim', 'Pam', 'Dwight', 'Jan', 'Tod..."
1,1,2,2,Diversity Day,Michael's off color remark puts a sensitivity ...,Ken Kwapis,B. J. Novak,2005-03-29,1002,6000000,...,23,29 March 2005,NaN,Ken Kwapis,B. J. Novak,46,217,3015,12,"'Michael', 'Mr. Brown', 'Jim', 'Dwight', 'Pam'..."
2,1,3,3,Health Care,Michael leaves Dwight in charge of picking the...,Ken Whittingham,Paul Lieberstein,2005-04-05,1006,5800000,...,22,5 April 2005,NaN,Ken Whittingham,Paul Lieberstein,39,244,2768,13,"'Michael', 'Pam', 'Jan', 'Jim', 'Dwight', 'Sta..."
3,1,4,4,The Alliance,"Just for a laugh, Jim agrees to an alliance wi...",Bryan Gordon,Michael Schur,2005-04-12,1004,5400000,...,23,12 April 2005,NaN,Bryan Gordon,Michael Schur,56,265,3163,14,"'Dwight', 'Michael', 'Stanley', 'Pam', 'Phylli..."
4,1,5,5,Basketball,Michael and his staff challenge the warehouse ...,Greg Daniels,Greg Daniels,2005-04-19,1005,5000000,...,23,19 April 2005,NaN,Greg Daniels,Greg Daniels,50,235,2551,20,"'Michael', 'Dwight', 'Jim', 'Phyllis', 'Pam', ..."


In [42]:
df['lista_personagens'][0]

"'Michael', 'Jim', 'Pam', 'Dwight', 'Jan', 'Todd Packer', 'Phyllis', 'Stanley', 'Oscar', 'Angela', 'Kevin', 'Ryan', 'Man', 'Roy'"

In [7]:
teste = lines.groupby(['temporada', 'episodio', 'cena'])['personagem'].max()
teste

temporada  episodio  cena
1          1         1        Michael
                     2        Michael
                     3            Pam
                     4        Michael
                     5         Dwight
                               ...   
9          23        112        Creed
                     113     Meredith
                     114      Phyllis
                     115          Jim
                     116          Pam
Name: personagem, Length: 8808, dtype: object

In [17]:
teste = teste.reset_index()
teste[teste['personagem']=='Michael']

,personagem,temporada,episodio,cena
407,Michael,11560,11560,11560
